In [1]:
cd C:\data\Human OC\CD data

C:\data\Human OC\CD data


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style

plt.style.use('ggplot')
import scipy

import seaborn as sns; sns.set(style='white')

import warnings
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina'

sns.set_context('talk', font_scale=0.8)
import statsmodels as sms
from statsmodels.stats import multitest
from statistics import mean


from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import decomposition
from sklearn.feature_selection import RFECV


In [4]:
df = pd.read_excel('metadata final selected patients.xlsx')
df.head()

,sample,stage_sorted,stage_early,type,type_sorted,stage,hospital,age,class_,group
0,OC41,Stage1,Early,ENDOMETRIOID,endometrioid,1c,Severance,42,OC,OC
1,OC4,Stage1,Early,CLEAR CELL,clear cell,1a,Severance,39,OC,OC
2,OC86,Stage1,Early,clear cell,clear cell,1A,Severance,39,OC,OC
3,OC123,Stage1,Early,Clear cell,clear cell,1A,Severance,43,OC,OC
4,OC82,Stage1,Early,Clear cell,clear cell,1a,Severance,44,OC,OC


In [5]:
other = df[(df['group'] == 'other')]
OC = df[(df['group'] == 'OC')]

<a id="1"></a>
### Significance test between ages of OC and non-OC patients

In [7]:
def metadata_Ttest(metadata, dfNon_OC, dfOC, alpha=0.05, var=True):
    
    statistic, pvalue = scipy.stats.ttest_ind(dfNon_OC[metadata], dfOC[metadata], 
                                              equal_var=var)
    
    for pval in pvalue:
        if pvalue > alpha:
            comment = 'Not significant'
        else:
            comment = 'Significant'
    comment
            
    Table = pd.DataFrame({'Confounder': metadata, 
                        'T-test p-value': pvalue, 'Significance':comment}) 
                             
    return Table

In [8]:
metadata = ['age'] 
metadata_Ttest(metadata,other,OC)

,Confounder,T-test p-value,Significance
0,age,0.124349,Not significant


In [9]:
print('Average age for OC patients is %.2f' %OC['age'].mean())
print('Average age for non-OC patients is %.2f' %other['age'].mean())

Average age for OC patients is 51.97
Average age for non-OC patients is 49.94


<a id="2"></a>
### Cohort characteristics 

In [10]:
print('Number of patients with Ovarian cancer 208\nNumber of patients with Non ovarian cancer malignancies is 117\nTotal number of patients after cohort matching is 325\n')
print(df.class_.value_counts())

Number of patients with Ovarian cancer 208
Number of patients with Non ovarian cancer malignancies is 117
Total number of patients after cohort matching is 325

OC     208
CC      46
BOT     41
BUT     28
B        2
Name: class_, dtype: int64


In [11]:
print(df.hospital.value_counts())

Severance    234
Dongsan       91
Name: hospital, dtype: int64


In [12]:
print('Number of OC patients at Severance and Dongsan hospital \n', OC.hospital.value_counts())

Number of OC patients at Severance and Dongsan hospital 
 Severance    141
Dongsan       67
Name: hospital, dtype: int64


In [13]:
print('Ovarian cancer pathological stage count:\n')
print(df.stage_sorted.value_counts())
#other = non OC malignancies 
#R = reccurence 

Ovarian cancer pathological stage count:

Stage3    88
Stage1    73
Stage2    20
Stage4    17
StageR    10
Name: stage_sorted, dtype: int64


In [14]:
print(OC.stage_early.value_counts())

Late          105
Early          93
reccurence     10
Name: stage_early, dtype: int64


In [15]:
print("Number of patients with early stage serous carcinoma is 29 \n")
print(OC.type_sorted.value_counts())

Number of patients with early stage serous carcinoma is 29 

serous            127
clear cell         32
Mucinous           19
endometrioid       18
MMMT                5
transitional        3
Adenocarcinoma      2
clear/serous        1
LGSC                1
Name: type_sorted, dtype: int64
